In [ ]:
"""
WeatherWise: A hands-on Python Weather Advisor Application

Features:
- Fetches real-time weather data and 5-day forecast using OpenWeatherMap API.
- Visualizes temperature and precipitation forecasts.
- Responds to natural language questions about weather.

Setup:
- Get your free OpenWeatherMap API key at https://openweathermap.org/api
- Replace the API_KEY variable with your API key.

Requirements:
- requests
- matplotlib
- numpy

Install dependencies:
pip install requests matplotlib numpy

Usage:
python weatherwise.py
"""

import requests
import matplotlib.pyplot as plt
import numpy as np
import datetime
import re

API_KEY = "YOUR_OPENWEATHERMAP_API_KEY"  # Replace this with your API key
BASE_WEATHER_URL = "https://api.openweathermap.org/data/2.5/weather"
BASE_FORECAST_URL = "https://api.openweathermap.org/data/2.5/forecast"

def kelvin_to_celsius(kelvin):
    return kelvin - 273.15

def get_weather_data(location, forecast_days=5):
    """
    Retrieve weather data for a specified location.

    Args:
        location (str): City or location name
        forecast_days (int): Number of days to forecast (1-5)

    Returns:
        dict: Weather data including current conditions and forecast
    """
    params = {
        'q': location,
        'appid': API_KEY
    }
    current_response = requests.get(BASE_WEATHER_URL, params=params)
    if current_response.status_code != 200:
        return {"error": current_response.json().get('message', '')}
    current_weather = current_response.json()

    forecast_response = requests.get(BASE_FORECAST_URL, params=params)
    if forecast_response.status_code != 200:
        return {"error": forecast_response.json().get('message', '')}
    forecast_data = forecast_response.json()

    # Limit forecast days (each day has 8 three-hourly forecasts)
    max_entries = forecast_days * 8
    forecast_data['list'] = forecast_data['list'][:max_entries]

    return {
        "current": current_weather,
        "forecast": forecast_data
    }

def parse_weather_question(question):
    """
    Parse a natural language weather question.

    Args:
        question (str): User's weather-related question

    Returns:
        dict: Extracted information including location, time period, and weather attribute
    """
    question = question.lower()
    parsed = {
        "location": None,
        "time_period": None,
        "attribute": None
    }

    # Extract location from question: Look for city keyword (assume last word if it's a city)
    # For simplicity, user location is fixed or none here as the app asks city at start
    # So we won't extract location from the question

    # Determine time period
    if "current" in question or "now" in question:
        parsed["time_period"] = "current"
    elif "tomorrow" in question:
        parsed["time_period"] = "tomorrow"
    elif "day after" in question or "two days" in question:
        parsed["time_period"] = "day_after_tomorrow"
    else:
        parsed["time_period"] = "forecast"

    # Determine attribute
    if "temperature" in question or "temp" in question:
        parsed["attribute"] = "temperature"
    elif "weather" in question or "condition" in question or "rain" in question or "snow" in question or "sunny" in question or "cloud" in question:
        parsed["attribute"] = "condition"
    elif "humidity" in question:
        parsed["attribute"] = "humidity"
    elif "wind" in question:
        parsed["attribute"] = "wind"
    elif "precipitation" in question or "rain" in question or "snow" in question:
        parsed["attribute"] = "precipitation"

    return parsed

def generate_weather_response(parsed_question, weather_data):
    """
    Generate a natural language response to a weather question.

    Args:
        parsed_question (dict): Parsed question data
        weather_data (dict): Weather data

    Returns:
        str: Natural language response
    """
    if "error" in weather_data:
        return f"Error fetching weather data: {weather_data['error']}"

    current_weather = weather_data["current"]
    forecast_data = weather_data["forecast"]
    location = current_weather.get('name', 'the location')

    # Helper to extract forecast entries by day offset
    def forecast_for_day(day_offset):
        now = datetime.datetime.now()
        target_date = now + datetime.timedelta(days=day_offset)
        entries = []
        for entry in forecast_data['list']:
            dt = datetime.datetime.fromtimestamp(entry['dt'])
            if dt.date() == target_date.date():
                entries.append(entry)
        return entries

    attr = parsed_question.get("attribute")
    time_period = parsed_question.get("time_period")

    if time_period == "current":
        if attr == "temperature":
            temp_c = kelvin_to_celsius(current_weather['main']['temp'])
            return f"The current temperature in {location} is {temp_c:.1f}°C."
        elif attr == "condition":
            cond = current_weather['weather'][0]['description']
            return f"The current weather in {location} is {cond}."
        elif attr == "humidity":
            humidity = current_weather['main']['humidity']
            return f"The current humidity in {location} is {humidity}%."
        elif attr == "wind":
            wind_speed = current_weather['wind']['speed']
            return f"The current wind speed in {location} is {wind_speed} m/s."
        elif attr == "precipitation":
            rain = current_weather.get('rain', {}).get('1h', 0)
            snow = current_weather.get('snow', {}).get('1h', 0)
            if rain > 0:
                return f"It is currently raining with {rain} mm precipitation in the last hour in {location}."
            elif snow > 0:
                return f"It is currently snowing with {snow} mm precipitation in the last hour in {location}."
            else:
                return f"There is no precipitation currently in {location}."
        else:
            cond = current_weather['weather'][0]['description']
            temp_c = kelvin_to_celsius(current_weather['main']['temp'])
            return f"The current weather in {location} is {cond} with temperature {temp_c:.1f}°C."

    elif time_period in ["tomorrow", "day_after_tomorrow", "forecast"]:
        day_offset = 1 if time_period == "tomorrow" else (2 if time_period == "day_after_tomorrow" else None)

        if time_period == "forecast":
            # For forecast, provide average temperature or general condition
            temps = []
            conditions = set()
            for entry in forecast_data['list']:
                temps.append(entry['main']['temp'])
                conditions.add(entry['weather'][0]['description'])
            avg_temp = kelvin_to_celsius(np.mean(temps))
            cond_str = ", ".join(list(conditions))
            return f"The forecast in {location} shows {cond_str} with an average temperature of {avg_temp:.1f}°C."

        entries = forecast_for_day(day_offset)
        if not entries:
            return f"Sorry, I do not have forecast data for {time_period} in {location}."

        if attr == "temperature":
            temps = [kelvin_to_celsius(e['main']['temp']) for e in entries]
            avg_temp = np.mean(temps)
            return f"The average temperature in {location} on {time_period.replace('_',' ')} will be {avg_temp:.1f}°C."
        elif attr == "condition":
            conditions = [e['weather'][0]['description'] for e in entries]
            unique_conditions = set(conditions)
            cond_str = ", ".join(unique_conditions)
            return f"The expected weather in {location} on {time_period.replace('_',' ')} is {cond_str}."
        elif attr == "humidity":
            humidities = [e['main']['humidity'] for e in entries]
            avg_humidity = np.mean(humidities)
            return f"The average humidity in {location} on {time_period.replace('_',' ')} will be {avg_humidity:.1f}%."
        elif attr == "wind":
            winds = [e['wind']['speed'] for e in entries]
            avg_wind = np.mean(winds)
            return f"The average wind speed in {location} on {time_period.replace('_',' ')} will be {avg_wind:.1f} m/s."
        elif attr == "precipitation":
            precipitations = []
            for e in entries:
                rain = e.get('rain', {}).get('1h', 0)
                snow = e.get('snow', {}).get('1h', 0)
                precipitations.append(rain + snow)
            total_precip = np.sum(precipitations)
            if total_precip > 0:
                return f"The total precipitation in {location} on {time_period.replace('_',' ')} will be {total_precip:.1f} mm."
            else:
                return f"No significant precipitation is forecast in {location} on {time_period.replace('_',' ')}."
        else:
            return f"I don't have information about {attr} for {time_period} in {location}."

    return "Sorry, I couldn't understand your question."

def create_temperature_visualisation(weather_data, output_type='display'):
    """
    Create visualisation of temperature data.

    Args:
        weather_data (dict): The processed weather data
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object
        Otherwise, displays the visualisation in the notebook
    """
    forecast_data = weather_data.get("forecast")
    if not forecast_data or "list" not in forecast_data:
        print("No forecast data to visualize.")
        return None

    times = []
    temps_c = []
    for entry in forecast_data['list']:
        dt = datetime.datetime.fromtimestamp(entry['dt'])
        temp_c = kelvin_to_celsius(entry['main']['temp'])
        times.append(dt)
        temps_c.append(temp_c)

    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(times, temps_c, marker='o', linestyle='-', color='tab:blue')
    ax.fill_between(times, temps_c, color='tab:blue', alpha=0.2)
    ax.set_title("Temperature Forecast", fontsize=16)
    ax.set_xlabel("Date and Time")
    ax.set_ylabel("Temperature (°C)")
    plt.xticks(rotation=45)
    ax.grid(True)
    plt.tight_layout()

    if output_type == 'display':
        plt.show()
    else:
        return fig

def create_precipitation_visualisation(weather_data, output_type='display'):
    """
    Create visualisation of precipitation data.

    Args:
        weather_data (dict): The processed weather data
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object
        Otherwise, displays the visualisation in the notebook
    """
    forecast_data = weather_data.get("forecast")
    if not forecast_data or "list" not in forecast_data:
        print("No forecast data to visualize.")
        return None

    times = []
    precipitation_mm = []

    for entry in forecast_data['list']:
        dt = datetime.datetime.fromtimestamp(entry['dt'])
        # Sum rain and snow (1 hour, fallback 0)
        rain = entry.get('rain', {}).get('1h', 0)
        snow = entry.get('snow', {}).get('1h', 0)
        total_precip = rain + snow
        times.append(dt)
        precipitation_mm.append(total_precip)

    fig, ax = plt.subplots(figsize=(12, 6))
    ax.bar(times, precipitation_mm, color='tab:blue', width=0.1)
    ax.set_title("Precipitation Forecast", fontsize=16)
    ax.set_xlabel("Date and Time")
    ax.set_ylabel("Precipitation (mm)")
    plt.xticks(rotation=45)
    ax.grid(True)
    plt.tight_layout()

    if output_type == 'display':
        plt.show()
    else:
        return fig

def main():
    print("Welcome to WeatherWise - Your Python Weather Advisor!")
    print("Please enter a city name to get started.")
    city = input("City: ").strip()

    weather_data = get_weather_data(city)
    if "error" in weather_data:
        print(f"Error: {weather_data['error']}")
        return

    # Show brief current weather summary
    current = weather_data['current']
    temp_c = kelvin_to_celsius(current['main']['temp'])
    condition = current['weather'][0]['description']
    humidity = current['main']['humidity']
    wind_speed = current['wind']['speed']
    print(f"\nCurrent weather for {current.get('name', city)}:")
    print(f"Temperature: {temp_c:.1f} °C")
    print(f"Condition: {condition}")
    print(f"Humidity: {humidity}%")
    print(f"Wind Speed: {wind_speed} m/s")

    # Create and display visualizations
    print("\nGenerating temperature forecast visualization...")
    create_temperature_visualisation(weather_data)

    print("Generating precipitation forecast visualization...")
    create_precipitation_visualisation(weather_data)

    print("\nYou can now ask me natural language questions about the weather.")
    print("Examples:")
    print(" - What's the current temperature?")
    print(" - What's the weather like tomorrow?")
    print(" - Will it rain tomorrow?")
    print("Type 'exit' to quit.\n")

    while True:
        question = input("Your question: ").strip()
        if question.lower() == 'exit':
            print("Goodbye! Stay safe and enjoy the weather.")
            break
        parsed = parse_weather_question(question)
        response = generate_weather_response(parsed, weather_data)
        print("Answer:", response)

if __name__ == "__main__":
    main()

